<h1>Email recipient recommendation</h1>

<i>Thomas Boudou, Guillaume Richard, Antoine Simoulin</i>

<p style="text-align: justify">It was shown that at work, employees frequently forget to include one or more recipient(s) before sending a message. Conversely, it is common that some recipients of a given message were actually not intended to receive the message. To increase productivity and prevent information leakage, the needs for effective <b>email recipient recommendation</b> systems are thus pressing.

In this challenge, you are asked to develop such a system, which, given the content and the date of a message, recommends a list of <b>10 recipients ranked by decreasing order of relevance</b>.</p>

In [1]:
# Requirements
%matplotlib inline
import random
import pandas as pd
import numpy as np
# do not display warnings
import warnings
warnings.filterwarnings("ignore")

# Functions files are saved in "src/" directory.
import sys
sys.path.append('src/')
from accuracy_measure import *

In [2]:
from load_data import *

# load files
# Data are saved in "data/" directory
path_to_data = '../data/'
training, training_info, test, test_info, y_df = load_data(path_to_data)

# create adress book
# /!\ can take 1-2 min
address_books = create_address_books(training, y_df)

# join train and test files
X_df = join_data(training_info, training)
X_sub_df = join_data(test_info, test)

In [3]:
import TFIDF_mod
from TFIDF_mod import TFIDF

# transform each mail body into tfidf vector
# /!\ function can take 1-2 min to execute
TFIDF = TFIDF()
X_TFIDF = TFIDF.fit_transform(X_df) # resulting shape : (43613, 275988)

--- 97 seconds ---


In [68]:
from sklearn.metrics.pairwise import cosine_similarity

def complete_prediction(k, sender, address_books, res_temp, K=10):
    # k the number of recipients to predict
    k_most = [elt[0] for elt in address_books[sender][:K] if elt not in res_temp]
    k_most = k_most[:k]
    if len(k_most) < k: # sender n'a pas assez de contacts
        k_most.extend([0] * (k-len(k_most)))
    return k_most

class Predictor_2():
    def __init__(self, X, y, sender, address_books,N=10):
        self.N = min(N,10)
        self.train = X
        self.predict = y.values
        self.sender = sender
        self.address_books = address_books
        self.X_recpt = {}
        
        # perform centroid Tf-Idf. i.e each 10 most frequent recipients is represented 
        # by an average of all mail he received.
        # exctract 10 most frequents recipients
        self.k_most = [elt[0] for elt in address_books[sender][:10]]
        # perform average Tf-Idf on 10 most frequents recipients
        for recpt in self.k_most: # loop trough 10 most frequents recipients
            self.X_recpt[recpt] = 0
            N = 0
            for i in range(X.shape[0]): # loop trough all mails send by sender
                if recpt in self.predict[i]: # if recipients is in mail
                    self.X_recpt[recpt] += X[i]
                N +=1
            self.X_recpt[recpt] /= N
            

    def predict_2(self, X):
        res = []
        cos = {}
        for i in range(X.shape[0]):
            # cosine similarity with 10 most frequents recpt
            for recpt, value in self.X_recpt.items():
                if isinstance(self.X_recpt[recpt], list): # check that sender has more than 10 recipients
                    cos[recpt] = cosine_similarity(X[i],self.X_recpt[recpt]).argsort()[:,0][0]
                else:
                    cos[recpt] = 1
            if self.N != 0:
                # return the 10 most frequent recipients in order 
                # given by similarity to their centroid Tf-Idf representation
                res_temp = list(dict(sorted(cos.items(), key=operator.itemgetter(1), reverse=True)[:10]))
            else:
                 res_temp = []
            # if less than 10 recipients, complete the prediction with more frequents users
            if len(res_temp) < 10:
                res_temp.extend(complete_prediction(10-len(res_temp),self.sender, self.address_books, res_temp))
            res.append(res_temp)
        return res

In [69]:
from sklearn.model_selection import ShuffleSplit

# splitting data for cross validation
skf = ShuffleSplit(n_splits=1, test_size=0.2)
print('%10s | %40s | %13s | %13s' %('sender_nb', 'sender', 'accuracy KNN','accuracy freq'))
print('%10s + %40s + %13s + %13s' %(10*'-', 40*'-', 13*'-', 13*'-'))

for train_is, test_is in skf.split(y_df):
    
    X_tfidf_train = X_TFIDF[train_is].copy()
    y_train = y_df.recipients.loc[train_is].copy()
    X_tfidf_test = X_TFIDF[test_is].copy()
    y_test = y_df.recipients.loc[test_is].copy()
    X_test_df = X_df.loc[test_is].copy()
    X_train_df = X_df.loc[train_is].copy()
    
    i=0
    pdt = {}
    pdt_freq = {}
    accuracy = {}
    accuracy_freq = {}
    accuracy_TOT = 0
    sender_test = X_test_df.sender.unique().tolist()
    y_pred = np.empty((X_test_df.shape[0],10),dtype=object)
    y_pred_freq = np.empty((X_test_df.shape[0],10),dtype=object)

    for sender in sender_test:
        print('%10s | %40s | ' %(sender_test.index(sender), sender), end='')
        # indices corresponding to the sender
        sender_train_is = np.array(X_train_df.sender == sender)
        sender_test_is = np.array(X_test_df.sender == sender)
        
        pdt[sender] = Predictor_2(X_tfidf_train[sender_train_is], y_train[sender_train_is], sender, address_books)
        pdt_freq[sender] = Predictor_2(X_tfidf_train[sender_train_is], y_train[sender_train_is], sender, address_books, N=0)
        y_pred[sender_test_is] = pdt[sender].predict_2(X_tfidf_test[sender_test_is])
        y_pred_freq[sender_test_is] = pdt_freq[sender].predict_2(X_tfidf_test[sender_test_is])
        
        accuracy[sender] = mapk(y_test[sender_test_is], y_pred[sender_test_is])
        accuracy_freq[sender] = mapk(y_test[sender_test_is], y_pred_freq[sender_test_is])
        
        accuracy_TOT += accuracy[sender]
        print('%13s | %13s' %(round(accuracy[sender],2),round(accuracy_freq[sender],2)))

    print('%30s'%(90*'-'))
    print('error TOT = %.2f' %(accuracy_TOT/len(accuracy)))

 sender_nb |                                   sender |  accuracy KNN | accuracy freq
---------- + ---------------------------------------- + ------------- + -------------
         0 |                  michelle.cash@enron.com |          0.07 |          0.21
         1 |                james.d.steffes@enron.com |          0.13 |          0.23
         2 |                  andrew.edison@enron.com |          0.42 |          0.45
         3 |                 barry.tycholiz@enron.com |          0.16 |          0.26
         4 |                     sally.beck@enron.com |          0.09 |          0.17
         5 |    schwabalerts.marketupdates@schwab.com |           1.0 |           1.0
         6 |                      eric.bass@enron.com |          0.15 |          0.19
         7 |               larry.f.campbell@enron.com |          0.15 |          0.18
         8 |                      m..forney@enron.com |           0.2 |          0.23
         9 |                    andy.zipper@enron.com 